<a id="import-data"></a>
## Import Data

# Machine Learning Project

## Table of Contents
- [Import Data](#import-data)
- [Data Exploration](#data-exploration)
  - [Categorical Features](#categorical-features)
  - [Numerical Features](#numerical-features)
  - [Plots](#plots)
- [Pre-processing](#pre-processing)
  - [Missing Values](#missing-values)
  - [String Distance](#string-distance)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# to calculate distance between strings
from thefuzz import process, fuzz

In [ ]:
df = pd.read_csv('data/train.csv').set_index('carID')
df.head()

In [ ]:
df.info()

#### Import Data Summary
- Dataset loaded successfully with `carID` as the index
- The dataset contains information about cars including both numerical features (price, mileage, tax, etc.) and categorical features (brand, model, transmission, etc.)
- Initial inspection shows multiple features that will require preprocessing:
  - Numerical features that need cleaning (negative values, outliers)
  - Categorical features that need standardization
  - Presence of missing values in several columns

<a id="data-exploration"></a>
## Data Exploration

<a id="categorical-features"></a>
### Categorical Features

#### Check Categorical Features Consistency

In [ ]:
num_duplicated_ids = df.index.duplicated().sum()
print(f'Number of duplicated carIDs: {num_duplicated_ids}')

In [ ]:
# List of categorical features
cat_cols = ['Brand', 'model', 'fuelType', 'transmission']

cat_outliers_examples = {col: df[col].value_counts().tail(10).index for col in cat_cols}

pd.DataFrame(cat_outliers_examples)

In [ ]:

df['hasDamage'].value_counts(dropna=False)

#### Categorical Features Summary
- Initial analysis reveals significant data quality issues across all categorical columns
- No standardization in categorical features, with multiple variations of the same values (different spellings, capitalizations)
- The `hasDamage` feature shows concerning data quality, containing only 0's and NA values
- Solution: We will implement string distance-based standardization using the `thefuzz` library to clean and standardize these features

<a id="numerical-features"></a>
### Numerical Features

In [ ]:
# List of numerical features
num_cols = [ 'mileage', 'tax', 'mpg', 'engineSize', 'year']


<a id="plots"></a>
### Plots

In [ ]:
num_cols = ['price', 'mileage', 'tax', 'mpg', 'engineSize', 'year']

plt.figure(figsize=(16, 10))
for i, col in enumerate(num_cols, 1):
    plt.subplot(2, 3, i)
    sns.boxplot(data=df, x=col, color="skyblue")
    plt.title(f"Boxplot of {col}", fontsize=11)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(16, 10))
for i, col in enumerate(num_cols, 1):
    plt.subplot(2, 3, i)
    sns.histplot(data=df, x=col, bins=30, kde=True, color="lightcoral")
    plt.title(f"Distribution of {col}", fontsize=11)
plt.tight_layout()
plt.show()

<a id="pre-processing"></a>
## Pre-processing

In [ ]:
X = df.drop(columns=["price"])   
y = df["price"]

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print(f"Training set size: {X_train.shape}")
print(f"Validation set size: {X_val.shape}")

In [ ]:
X_train_clean = X_train.copy()
X_val_clean = X_val.copy()

<a id="missing-values"></a>
### Missing Values

In [ ]:
num_general = ["tax", "previousOwners", "paintQuality%", "engineSize"]

median_imputer = SimpleImputer(strategy="median")
X_train_clean[num_general] = median_imputer.fit_transform(X_train_clean[num_general])
X_val_clean[num_general] = median_imputer.transform(X_val_clean[num_general])

In [ ]:
if "mpg" in X_train_clean.columns and "fuelType" in X_train_clean.columns:
    mpg_group_medians = X_train_clean.groupby("fuelType")["mpg"].median()

    def impute_mpg(row):
        if pd.isnull(row["mpg"]):
            return mpg_group_medians.get(row["fuelType"], X_train_clean["mpg"].median())
        return row["mpg"]

    X_train_clean["mpg"] = X_train_clean.apply(impute_mpg, axis=1)
    X_val_clean["mpg"] = X_val_clean.apply(impute_mpg, axis=1)

In [ ]:
cat_general = ["Brand", "model", "transmission", "fuelType"]

mode_imputer = SimpleImputer(strategy="most_frequent")
X_train_clean[cat_general] = mode_imputer.fit_transform(X_train_clean[cat_general])
X_val_clean[cat_general] = mode_imputer.transform(X_val_clean[cat_general])

In [ ]:
if "hasDamage" in X_train_clean.columns:
    X_train_clean["hasDamage"] = X_train_clean["hasDamage"].fillna("Unknown")
    X_val_clean["hasDamage"] = X_val_clean["hasDamage"].fillna("Unknown")

In [ ]:
missing_summary_train = X_train_clean.isnull().sum()
missing_summary_val = X_val_clean.isnull().sum()

print("Remaining missing values (train):", missing_summary_train.sum())
print("Remaining missing values (validation):", missing_summary_val.sum())

In [ ]:
remaining_missing = X_train_clean.isnull().sum()
remaining_missing = remaining_missing[remaining_missing > 0]
print("Columns with remaining NaN values:")
display(remaining_missing)

In [ ]:
for col in ["year", "mileage", "mpg"]:
    if col in X_train_clean.columns:
        median_value = X_train_clean[col].median()
        X_train_clean[col].fillna(median_value, inplace=True)
        X_val_clean[col].fillna(median_value, inplace=True)

remaining_missing = X_train_clean.isnull().sum()
remaining_missing = remaining_missing[remaining_missing > 0]
print("Columns with remaining NaN values after final cleaning:")
display(remaining_missing)

In [ ]:
remaining_missing = X_train_clean.isnull().sum()
remaining_missing = remaining_missing[remaining_missing > 0]
print("Columns with remaining NaN values:")
display(remaining_missing)

### Numeric Features

- carID
- year
- price
- mileage
- tax
- mpg
- engineSize
- paintQuality
- previousOwners
- hasDamage


In [ ]:
#Check if there are negative features that should not be negative
numeric_features = X_train_clean.select_dtypes(include=['int64', 'float64']).columns
negative_values = {}
for col in numeric_features:
    negative_count = (X_train_clean[col] < 0).sum()
    if negative_count > 0:
        negative_values[col] = negative_count   

for v in negative_values:
    print(f"Feature '{v}' has {negative_values[v]} negative values.")

### Strategy:

- Change negative values to `NaN`
- Remove extreme outliers
- Impute using the appropriate method (median, mode, or group-based)
- Convert the column back to integer type if applicable

Note: Still have to choose what method to use to change the missing values in the feature "hasDamage" before I can switch from float to int

In [ ]:
cols_to_fix = list(negative_values.keys())

for feature in cols_to_fix:
    X_train_clean.loc[X_train_clean[feature] < 0, feature] = np.nan
    X_val_clean.loc[X_val_clean[feature] < 0, feature] = np.nan


In [ ]:
for col in X_train_clean.select_dtypes(include=['int64', 'float64']).columns:
    q1 = X_train_clean[col].quantile(0.25)
    q3 = X_train_clean[col].quantile(0.75)
    iqr = q3 - q1
    lower_lim = q1 - (1.5 * iqr)
    upper_lim = q3 + (1.5 * iqr)
    X_train_clean[col] = X_train_clean[col].mask((X_train_clean[col] < lower_lim) | (X_train_clean[col] > upper_lim), np.nan)
    X_val_clean[col] = X_val_clean[col].mask((X_val_clean[col] < lower_lim) | (X_val_clean[col] > upper_lim), np.nan)


In [ ]:
for feature in cols_to_fix:
    median_value = X_train_clean[feature].median()
    X_train_clean[feature].fillna(median_value, inplace=True)
    X_val_clean[feature].fillna(median_value, inplace=True)


In [ ]:
int_cols = ['year', 'previousOwners']

median_year = X_train_clean['year'].median()
X_train_clean['year'].fillna(median_year, inplace=True)
X_val_clean['year'].fillna(median_year, inplace=True)

for feature in int_cols:
    X_train_clean[feature] = X_train_clean[feature].astype(int)
    X_val_clean[feature] = X_val_clean[feature].astype(int)

In [ ]:
(X_train_clean["year"] < 1950).sum() and (X_train_clean["year"] > 2025).sum()

In [ ]:
X_train_clean.head()


In [ ]:
X_val_clean.head()

Final check for nan values

In [ ]:
X_train_clean.isnull().sum()

Check for other strange values

In [ ]:
print(f"Number of percentages above 100: {(X_train_clean["paintQuality%"] > 100).sum()}")

In [ ]:
X_train_clean.loc[(X_train_clean['paintQuality%'] > 100), 'paintQuality%'] = np.nan
X_val_clean.loc[(X_val_clean['paintQuality%'] > 100), 'paintQuality%'] = np.nan

median_paint = X_train_clean['paintQuality%'].median()
X_train_clean['paintQuality%'].fillna(median_paint, inplace=True)
X_val_clean['paintQuality%'].fillna(median_paint, inplace=True)



In [ ]:
missing_counts = X_train_clean.isnull().sum()
missing_percent = (missing_counts / len(df)) * 100

missing_summary = pd.DataFrame({
    "Missing Count": missing_counts,
    "Missing %": missing_percent.round(2)
}).sort_values(by="Missing %", ascending=False)

missing_summary[missing_summary["Missing Count"] > 0]

In [ ]:
num_cols = X_train_clean.select_dtypes(include=["int64", "float64"]).columns

outlier_summary = []

for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = X_train_clean[(X_train_clean[col] < lower_bound) | (X_train_clean[col] > upper_bound)]
    
    outlier_summary.append({
        "Feature": col,
        "Lower Bound": round(lower_bound, 2),
        "Upper Bound": round(upper_bound, 2),
        "Outlier Count": len(outliers),
        "Outlier %": round(len(outliers) / len(df) * 100, 2)
    })

outlier_df = pd.DataFrame(outlier_summary).sort_values(by="Outlier %", ascending=False)
outlier_df

<a id="string-distances"></a>
# Calculating the string distances
to standardize the column strings that have misspelled names using the library "thefuzz"

First we are standarlizing the brands

In [ ]:
X_train_clean

In [ ]:

brands = ["volkswagen", "ford", "fiat", "volksvagen", "wolkswagen"]


In [ ]:
BRAND_COL = "Brand"   
ANCHOR_NUM = 9            # top 10 k most frequent ancor
SIM_THRESHOLD = 85

# threshold for fuzzy matching
SIM_THRESHOLD = 85

In [ ]:
_brand_norm = X_train_clean[BRAND_COL].astype(str)   # make it shure that is a string
_brand_norm = _brand_norm.str.strip()                # remove extra spaces
_brand_norm = _brand_norm.str.lower()                # to lower case

df["Brand"].str.lower().value_counts() #analysing the most frequent brand names in our datasety

Now we get the most common brand names as we analysed before and create a map dictionary for corrections

In [ ]:
anchors = _brand_norm.value_counts().head(ANCHOR_NUM).index.tolist()
anchors

In [ ]:


mapping = {}

# for each unique brand, find the closest anchor and map if similarity is high
for val in _brand_norm.unique():
    if val in anchors:
        continue  # already a good brand
    match, score = process.extractOne(val, anchors, scorer=fuzz.token_set_ratio)
    if score >= SIM_THRESHOLD:
        mapping[val] = match  # map wrong/rare brand to correct one

In [ ]:
X_train_clean["brand_clean"] = _brand_norm.map(lambda x: mapping.get(x, x))

In [ ]:
mapping

In [ ]:
X_train_clean["brand_clean"]